In [ ]:
import nltk
import csv

Посмотрим на данные: это отзывы о ресторанах и оценка. Будем решать многоклассовую классификацию

In [ ]:
! head -n 2 train.data

In [ ]:
! wc -l train.data

Считаем выборку, поделим на трейн и тест так, чтобы в x_train был raw text

In [ ]:
train_file = #
train_set = #

train_data, train_label = # 
from sklearn.cross_validation import train_test_split

x_train, x_validate, y_train, y_validate = train_test_split(train_data, train_label, test_size=0.2, random_state=0)

In [ ]:
x_train[0]

In [ ]:
y_train[0]

"Тупое" решение:

Посмотрим что будет, если применить самое простое решение: найти 100 самых частотных слов и использовать их в качестве признаков.

- Найдем топ 100 самых частотных слов

In [ ]:
from collections import Counter


In [ ]:
most_frequent_word = #

В наивный байесовский классификатор надо подать текст в определенном формате

In [ ]:
def make_bow_sample(bow, sample):
    for s in sample:
        s = s.strip().split()
        yield { word:word in s for word, _ in bow}

In [ ]:
bow_train = [(x, y) for x, y in zip(make_bow_sample(most_frequent_word, x_train), y_train)]
bow_validate = [x for x in make_bow_sample(most_frequent_word, x_validate)]

In [ ]:
bow_train[0]

Воспользуемся наивным байесовским классификатором. 
Плюс данного классификатора - можно посмотреть какиме слова оказались наиболее полезными.

In [ ]:
nb = nltk.NaiveBayesClassifier.train(bow_train)
print(nb.show_most_informative_features())
predicted = [nb.classify(o) for o in bow_validate]

In [ ]:
import numpy as np

Замерьте качество с помощью accuracy

In [ ]:
print 'accuracy', #

In [ ]:
import csv
import pandas as pd
from sklearn.cross_validation import train_test_split
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier
import numpy as np
import glob

In [ ]:
y_train = map(float, y_train)
y_validate = map(float, y_validate)

Обучите TfidfVectorizer на трейне, примените на тесте.

Почему надо действовать именно так, почему нельзя обучить и на трейне, и на тесте?

In [ ]:
tfidf = 
Xtrain = 
Xtest = 

- Попробуйте LodisticRegression, LinearSVC, SGDClassifier с какой-нибудь функцией потерь.
- При обучении  SGDClassifier не забудьте поставить побольше итераций, так как это итеративный метод
- параметр class_weight='balanced' может быть полезен. Что он означает?
- Можете повариьировать так же параметры TF-IDF vectorizer
- Попробовать прологарифмировать частоты np.log1p(), или другое нелинейное преобразование.

SVM vs LinearSVC
(LinearSVC быстрее, но не выадет вероятностей, а лишь расстояние до решающей границы. Перевести в вероятности можно, откалибровав http://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html)


In [ ]:
lr = LogisticRegression(C=, n_jobs=-1)
lr.fit(Xtrain, y_train)
lr_pr = 

Замерьте качество

In [ ]:
print 'accuracy', 

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
# clf = [LogisticRegression(n_jobs=-1)
# LinearSVC(C=1, loss='hinge', class_weight='balanced'),
# SGDClassifier(loss='modified_huber', class_weight='balanced', alpha=1e-2, n_iter=50, n_jobs=-1),
# SGDClassifier(loss='squared_hinge', class_weight='balanced', alpha=1e-2, n_iter=50, n_jobs=-1),
# SGDClassifier(loss='hinge',class_weight='balanced', alpha=1e-2, random_state=3, n_iter=50, n_jobs=-1)]

Какой алгоритм сработал лучше свего?

Когда обучаем многоклассовую классификацию для такой задачи, не учитываем то, что метки 1 и 2 более похожи между собой, чем 4 и 5. Как это можно было бы учесть при обучении модели?

Переходим к нейросетевым подходам

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from keras.preprocessing.text import Tokenizer

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from keras.utils import np_utils
import numpy as np

In [ ]:
x_train[0]

In [ ]:
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(x_train)
tokenizer

In [ ]:
sequences = tokenizer.texts_to_sequences(x_train)
X_train = tokenizer.sequences_to_matrix(sequences, mode='count')
sequences = tokenizer.texts_to_sequences(x_validate)
X_test = tokenizer.sequences_to_matrix(sequences, mode='count')

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train = np_utils.to_categorical(y_train)[:, 1:]
y_test = np_utils.to_categorical(y_validate)[:, 1:]

In [ ]:
model = Sequential()
model.add(Dense(64, input_shape=(MAX_FEATURES,), activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(5, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=nb_epoch, batch_size=batch_size,  validation_split=0.1)

In [ ]:
pr = model.predict(X_test)

Замерьте качество

Теперь пробуем LSTM

In [ ]:
from keras.preprocessing import sequence
sequences = tokenizer.texts_to_sequences(x_train)
X_train = sequence.pad_sequences(sequences, maxlen=TEXT_LENGTH)
sequences = tokenizer.texts_to_sequences(x_validate)
X_test = sequence.pad_sequences(sequences, maxlen=TEXT_LENGTH)

In [ ]:
max_review_length = max([len(el) for el in X_train])
top_words = 5000

In [ ]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(5, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

In [ ]:
pr = model.predict(X_test)

Замерьте качество

Теперь пробуем добавить сверточный слой

In [ ]:
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)

In [ ]:
pr = model.predict(X_test)

Замерьте качество